# Lab 1: Getting to know TensorFlow

Na tomto cvičení si zopakujeme základné poznatky ktoré ste získali na predmete Neurónové siete. Vysvetlíme si, ako funguje perceptrón, čo sú aktivačné funkcie a načo slúžia, ako sa trénujú neurónové siete. Následne sa oboznámime so základnými funkciami a prvkami softvérového rámca TensorFlow.

**Zdroje**

[Introduction to Deep Learning 1. prednáška](http://introtodeeplearning.com/materials/2019_6S191_L1.pdf)

[Dokumentácia TensorFlow](https://www.tensorflow.org/api_docs)

## 0. Importovanie TensorFlow

Na to, aby sme vedeli zadefinovať naše prvé neurónové siete, je potrebné mať nainštalovaný a načítaný softvérový rámec TensorFlow. Návod na inštaláciu nájdete v [Lab 0](https://github.com/ianmagyar/dl-course/blob/master/labs/lab0-getting-ready.md).

Ak už máte nainštalovaný TensorFlow, neostáva Vám nič iné, ako ho načítať:

In [ ]:
import tensorflow as tf
tf.enable_eager_execution()

Pri Eager vykonávaní sa operácie vykonajú okamžite pri volaní z Pythonu, čo nám umožňuje rýchle debugovanie a práve preto vám ho odporúčame používať pri vytvoraní vašich riešení.

## 1. Výpočty v TensorFlowe

Pred tým než vytvoríme naše prvé neurónové siete v TensorFlowe, oboznámime sa so základnými výpočtami. Názov TensorFlow popisuje spôsob vykonávania výpočtov v tomto softvérovom rámci. Tensory sú vlastne údaje (hodnoty alebo viacdimenzionálne polia) a výpočty predstavujú *flow* týchto dát. Na začiatok zadefinujeme jednoduchú operáciu sčítania pomocou TensorFlow:

![](figures/lab1-addition.png)

In [ ]:
# create the nodes in the graph, and initialize values
a = tf.constant(13, name="a")
b = tf.constant(37, name="b")

# add together the two values
c = tf.add(a, b, name="c")
print(c)

Na základe predošlého príkladu vytvorte trošku viac zložitý graf:

![](figures/lab1-complicated-graph.png)

In [ ]:
# create the nodes in the graph, and initialize values
a = tf.constant(2.5, name="a")
b = tf.constant(6.5, name="b")

c = tf.add(a, b, name="c")
d = tf.subtract(b, 1, name="d")
e = tf.multiply(c, d, name="e")

print(e)

## 2. Perceptrón

Perceptrón je neurón navrhnutý Frankom Rosenblattom, ktorý predstavuje základný výpočtový prvok neurónových sietí. Štruktúru resp. topológiu perceptrónu vidíte na obrázku.

![Štruktúra perceptrónu](figures/lab1-perceptron.png)

Výpočet v perceptróne sa pozostáva z váženej sumy, pripočítaní biasu a aplikácie aktivačnej funkcie. Pri vytvorení robustných neurónových sietí hlbokého učenia budeme používať Keras API, ale v tomto kroku vytvoríme jednoduchý perceptrón pomocou základnych metód TensorFlow. Aby sme si vedeli skontrolovať, či naše riešenie funguje správne, zadefinujeme nielen vstupy, ale aj váhy.

In [ ]:
# simple perceptron with two input nodes
def my_perceptron(x):
    # define some arbitrary weights for the two input values
    W = tf.constant([[3, -2]], shape=(1, 2), dtype=tf.float32)

    # define the bias of the perceptron
    b = 1
    
    # compute weighted sum (hint: check out tf.matmul)
    z = tf.matmul(x, W, transpose_b=True) + b

    # apply the sigmoid activation function (hint: use tf.sigmoid)
    output = tf.sigmoid(z)

    return output

sample_input = tf.constant([[-1, 2]], shape=(1, 2), dtype=tf.float32)

# if you've done everything correctly, this should give you a tensor with value 0.002
result = my_perceptron(sample_input)
print(result)

## 3. Neurónové siete

Ak chceme naše siete natrénovať, konštantné váhy nám nepomôžu a práve preto si teraz opravím predchádzajúcí kód tak, aby sme výsledný model vedeli natrénovať. Zároveň, náš model rozšírime o niekoľko neurónov, tak aby sme dostali *fully connected* (*dense*) vrstvu.

In [ ]:
# x: input values
# n_in: number of input nodes
# n_out: number of output nodes
def my_dense_layer(x, n_in, n_out):
    # define variable weights as a matrix and biases
    # initialize weights for one
    # initialize biases for zero
    W = tf.Variable(tf.ones((n_in, n_out)))
    b = tf.Variable(tf.zeros((1, n_out)))
    
    # compute weighted sum (hint: check out tf.matmul)
    z = tf.matmul(x, W) + b

    # apply the sigmoid activation function (hint: use tf.sigmoid)
    output = tf.sigmoid(z)

    return output

Ako aj pred tým, naše riešenie vieme otestovať zadaním ľubovoľných hodnôt (s dodržaním počtu vstupných a výstupných neurónov).

In [ ]:
sample_input = tf.constant([[1, 2.]], shape=(1, 2))
print(my_dense_layer(sample_input, n_in=2, n_out=3))

Síce definícia jednej vrstvy nie je až také problemtické, pri vytvorení hlbokých sietí by sme sa veľmi rýchlo narazili na problém. Práve preto sa pozrieme na to, ako by sme vedeli vytvoriť rovnaký model pomocou Keras API.

Keras nám ponúka niekoľko pripravených štandardných riešení, z ktorých my teraz použijeme [Dense](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Sequential) vrstvu a [Sequential](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Sequential) model.

In [ ]:
# import relevant packages
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# define the number of input and output nodes
n_input_nodes = 2
n_output_nodes = 3

# first define the model
model = Sequential()

# define a dense layer based on weights and biases
dense_layer = Dense(n_output_nodes, input_shape=(n_input_nodes, ), activation='sigmoid')

model.add(dense_layer)

x_input = tf.constant([[1, 2.]], shape=(1, 2))

# feed the input into the model and predict the output
print(model.predict(x_input))

# 4. Trénovanie neurónovej siete

Trénovanie neurónovej siete sa najčastejšie robí pomocou gradient descent, proces, ktorý úspešne hľadá minimum v *n+1* dimenzionálnom prostredí, kde *n* je počet váh a dimenzia +1 vyjadruje chybu pri danej konfigurácii váh. Vďaka Eager vykonávania vieme tento proces vizualizovať na príklade derivácie jednoduchej funkcie *y = (x - 1)<sup>2</sup>*.

In [ ]:
import matplotlib.pyplot as plt

x = tf.Variable([tf.random.normal([1])])
print("Initializing x={}".format(x.numpy()))
learning_rate = 1e-2
history = []

for i in range(500):
    with tf.GradientTape() as tape:
        y = (x - 1)**2 # record the forward pass on the tape

        grad = tape.gradient(y, x) # compute the gradient of y with respect to x
        new_x = x - learning_rate*grad # sgd update
        x.assign(new_x) # update the value of x
        history.append(x.numpy()[0])

plt.plot(history)
plt.plot([0, 500],[1,1])
plt.legend(('Predicted', 'True'))
plt.xlabel('Iteration')
plt.ylabel('x value')
plt.show()